In [2]:
import tensorflow as tf
from transformers import BertTokenizer
import pandas as pd
import numpy as np
mdir = 'models/'
ddir = 'data/'

In [3]:
model = tf.keras.models.load_model(mdir+'sentiment_model_v1', compile = False)

2023-03-22 17:13:58.693965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-22 17:14:00.785496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-22 17:14:00.785889: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-22 17:14:00.844712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5, weight_decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer = optimizer, loss = loss, metrics = [acc])

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [8]:
def prep_data(text):
    tokens = tokenizer(text, max_length = 512, truncation = True,
                      padding = 'max_length', add_special_tokens = True,
                      return_tensors = 'tf')
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

In [9]:
probs = model.predict(prep_data('hello world'))[0]
probs

1/1 [==============================] - 10s 10s/step


array([0.00521353, 0.03077668, 0.7011549 , 0.22871119, 0.03414375],
      dtype=float32)

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
df = pd.read_csv(ddir+'test.tsv', sep = '\t')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [12]:
df = df.drop_duplicates(subset = ['SentenceId'], keep = 'first')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
15,156076,8546,"Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world ."
93,156154,8547,Once you get into its rhythm ... the movie becomes a heady experience .
117,156178,8548,I kept wishing I was watching a documentary about the wartime Navajos and what they accomplished instead of all this specious Hollywood hoo-ha .
158,156219,8549,"Kinnear does n't aim for our sympathy , but rather delivers a performance of striking skill and depth ."


In [13]:
df['Sentiment'] = None

In [14]:
for i, row in df.iterrows():
    tokens = prep_data(row['Phrase'])
    probs = model.predict(tokens)
    pred = np.argmax(probs)
    df.at[i, 'Sentiment'] = pred

1/1 [==============================] - 0s 101ms/step


1/1 [==============================] - 0s 120ms/step


1/1 [==============================] - 0s 123ms/step


1/1 [==============================] - 0s 106ms/step


1/1 [==============================] - 0s 121ms/step


...

1/1 [==============================] - 0s 116ms/step


1/1 [==============================] - 0s 112ms/step


1/1 [==============================] - 0s 104ms/step


1/1 [==============================] - 0s 116ms/step


1/1 [==============================] - 0s 124ms/step


In [15]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine effort .,2
15,156076,8546,"Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world .",3
93,156154,8547,Once you get into its rhythm ... the movie becomes a heady experience .,4
117,156178,8548,I kept wishing I was watching a documentary about the wartime Navajos and what they accomplished instead of all this specious Hollywood hoo-ha .,0
158,156219,8549,"Kinnear does n't aim for our sympathy , but rather delivers a performance of striking skill and depth .",4


In [16]:
df.tail()

,PhraseId,SentenceId,Phrase,Sentiment
66220,222281,11851,Not sweet enough to liven up its predictable story and will leave even fans of hip-hop sorely disappointed .,1
66239,222300,11852,"Nasty , ugly , pointless and depressing , even if you hate clowns .",0
66253,222314,11853,"With tightly organized efficiency , numerous flashbacks and a constant edge of tension , Miller 's film is one of 2002 's involvingly adult surprises .",4
66280,222341,11854,They should have called it Gutterball .,2
66287,222348,11855,"A long-winded , predictable scenario .",1
